In [37]:
import numpy as np
import pickle
import utils
import time
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tqdm import tqdm
from collections import Counter
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
emoji_vectors = pickle.load(open('data/emoji_vectors.p', 'rb'))
moby_dick_vectors = pickle.load(open('data/moby_dick_vectors.p', 'rb'))
moby_dick_sents = pickle.load(open('data/moby_dick_sents.p', 'rb'))
raw_train = pickle.load(open('models/raw_train.p', 'rb'))
raw_test = pickle.load(open('models/raw_test.p', 'rb'))

emoji_embedding = np.array([v for v in emoji_vectors.values()])

In [27]:
tf.reset_default_graph()
sess.close()
sess = tf.InteractiveSession()

In [28]:
batch_size = 128
nodes = 300
embed_size = 300
x_seq_length = 32

inputs = tf.placeholder(tf.float32, (None, x_seq_length, embed_size), 'inputs')
input_mean = tf.nn.l2_normalize(tf.reduce_mean(inputs, axis=1), axis=1, name='input_mean')

output_embedding = tf.constant(emoji_embedding, name='output_embedding')

with tf.name_scope('network'):
    cell_fw = tf.contrib.rnn.LSTMCell(nodes)
    cell_bw = tf.contrib.rnn.LSTMCell(nodes)

    ((fw_out, bw_out) , (_, __)) = tf.nn.bidirectional_dynamic_rnn(cell_fw=cell_fw,
                                                    cell_bw=cell_bw, inputs=inputs, dtype=tf.float32)
    lstm_outputs = tf.add(fw_out, bw_out)
    
    logits = tf.layers.dense(lstm_outputs, units=len(emoji_vectors), activation='softmax', name='dense') 
    outputs = utils.matmul3d(logits, output_embedding)

    output_mean = tf.nn.l2_normalize(tf.reduce_mean(outputs, axis=1), axis=1)

with tf.name_scope("optimization"):
    loss = tf.losses.cosine_distance(input_mean, output_mean, axis=1)
    optimizer = tf.train.AdamOptimizer(1e-3).minimize(loss)
    
tf.summary.scalar('loss', loss)
merged = tf.summary.merge_all()
writer = tf.summary.FileWriter('models/blstm_moby_dick/1', sess.graph)

In [14]:
train_sents, test_sents = train_test_split(moby_dick_sents, random_state=42)

In [15]:
X_train = [[moby_dick_vectors[w] for w in s] for s in train_sents]
X_test = [[moby_dick_vectors[w] for w in s] for s in test_sents]

In [32]:
sess.run(tf.global_variables_initializer())
epochs = 100
start = time.time()

for i in range(epochs):
    t = time.time()
    losses = []
    for X in utils.batch_generator(X_train, batch_size):
        _, l, summary = sess.run([optimizer, loss, merged], feed_dict={inputs:X})
        losses.append(l)
    writer.add_summary(summary, global_step=i)
#     if l < .0005:
#         print('Epoch {:3} Loss: {:>6.3f} Epoch duration: {:>6.3f}s'.format(i, l, time.time() - t))
#         break
#     elif not i%10:
    print('Epoch {:3} Average Loss: {:>6.3f} Epoch duration: {:>6.3f}s'.format(i, np.mean(losses, axis=-1), time.time() - t))

saver = tf.train.Saver()
saver.save(sess, 'models/blstm_moby_dick/model1')
print('Total training time:', time.time()-start)

predictions = []
losses = []
emoji_keys = list(emoji_vectors.keys())
for x in tqdm(X_test):
    lo, l = sess.run([logits, loss], feed_dict={inputs:np.array(x).reshape(-1, 32, 300)})
    pred = np.argmax(lo, axis=2).reshape(32,)
    predictions.append([emoji_keys[i] for i in pred])
    losses.append(l)

print('Average test loss:', np.mean(losses, axis=-1))
print()

for i in range(10):
    print('Test sentence:', ' '.join(w for w in test_sents[i] if w))
    print('Prediction:', set(predictions[i]))
    print('Cosine distance:', losses[i])
    print()

Epoch   0 Average Loss:  0.403 Epoch duration: 43.788s
Epoch   1 Average Loss:  0.376 Epoch duration: 43.765s
Epoch   2 Average Loss:  0.358 Epoch duration: 44.040s
Epoch   3 Average Loss:  0.344 Epoch duration: 44.457s
Epoch   4 Average Loss:  0.335 Epoch duration: 43.573s
Epoch   5 Average Loss:  0.329 Epoch duration: 43.805s
Epoch   6 Average Loss:  0.324 Epoch duration: 43.804s
Epoch   7 Average Loss:  0.320 Epoch duration: 43.626s
Epoch   8 Average Loss:  0.317 Epoch duration: 43.628s
Epoch   9 Average Loss:  0.314 Epoch duration: 44.057s
Epoch  10 Average Loss:  0.312 Epoch duration: 44.330s
Epoch  11 Average Loss:  0.310 Epoch duration: 43.570s
Epoch  12 Average Loss:  0.308 Epoch duration: 43.638s
Epoch  13 Average Loss:  0.307 Epoch duration: 43.552s
Epoch  14 Average Loss:  0.306 Epoch duration: 44.017s
Epoch  15 Average Loss:  0.305 Epoch duration: 44.081s
Epoch  16 Average Loss:  0.304 Epoch duration: 44.312s
Epoch  17 Average Loss:  0.303 Epoch duration: 44.558s
Epoch  18 

  0%|          | 4/2515 [00:00<01:05, 38.61it/s]

Total training time: 4377.371751785278


100%|██████████| 2515/2515 [00:30<00:00, 82.66it/s]

Average test loss: 0.3024252

Test sentence: an old pike head sir there were seams dents in it
Prediction: {'🖊', '⁉', '👴', '🗑', '🤕', '🈁', '🕳', '🐆'}
Cosine distance: 0.2173863

Test sentence: this one poor hunt then the best lance out all surely he will not hang back when every foremast hand has clutched whetstone
Prediction: {'🖊', '🗑', '🔂', '🈁', '🐆', '👏', '🙏'}
Cosine distance: 0.12835592

Test sentence: drop them over fore aft
Prediction: {'🐆', '🖊', '⚜'}
Cosine distance: 0.34018457

Test sentence: in the infancy the first settlement the emigrants were several times saved from starvation by the benevolent biscuit the whale ship luckily dropping an anchor in their waters
Prediction: {'🖊', '🍠', '🈺', '0', '⚰', '🆙', '⚜', '🖤', '🦏', '🐋', '🕜', '🌊', '🐆', '🛬', '⏮', '💯', '🈶', '〽', '💏'}
Cosine distance: 0.16908199

Test sentence: mighty whales which swim in sea water have sea oil swimming in them
Prediction: {'🖊', '🗑', '🐋', '🤼', '🌊', '🈴', '🐆', '⛽'}
Cosine distance: 0.14456332

Test sentence: round

In [38]:
saver = tf.train.Saver()
saver.restore(sess, 'models/blstm_moby_dick/model1')

INFO:tensorflow:Restoring parameters from models/seq2seq_moby_dick/model1


In [39]:
predictions = []
losses = []
emoji_keys = list(emoji_vectors.keys())
for x in tqdm(X_test):
    lo, l = sess.run([logits, loss], feed_dict={inputs:np.array(x).reshape(-1, 32, 300)})
    pred = np.argmax(lo, axis=2).reshape(32,)
    predictions.append([emoji_keys[i] for i in pred])
    losses.append(l)



  0%|          | 0/2515 [00:00<?, ?it/s]

  0%|          | 5/2515 [00:00<00:56, 44.40it/s]

  0%|          | 10/2515 [00:00<00:56, 44.48it/s]

  1%|          | 15/2515 [00:00<00:55, 44.80it/s]

  1%|          | 20/2515 [00:00<00:55, 45.16it/s]

  1%|          | 25/2515 [00:00<00:54, 45.91it/s]

  1%|          | 30/2515 [00:00<00:53, 46.04it/s]

  1%|▏         | 35/2515 [00:00<00:53, 46.07it/s]

  2%|▏         | 40/2515 [00:00<00:53, 46.22it/s]

  2%|▏         | 45/2515 [00:00<00:53, 46.30it/s]

  2%|▏         | 50/2515 [00:01<00:53, 46.36it/s]

  2%|▏         | 55/2515 [00:01<00:53, 46.34it/s]

  2%|▏         | 60/2515 [00:01<00:53, 46.06it/s]

  3%|▎         | 65/2515 [00:01<00:53, 46.01it/s]

  3%|▎         | 70/2515 [00:01<00:53, 45.84it/s]

  3%|▎         | 75/2515 [00:01<00:53, 45.90it/s]

  3%|▎         | 80/2515 [00:01<00:52, 46.52it/s]

  3%|▎         | 85/2515 [00:01<00:52, 46.43it/s]

  4%|▎         | 90/2515 [00:01<00:52, 46.22it/s]

  4%|▍         | 95/2515 [00:02<00:52, 

In [33]:
sorted_losses = np.argsort(losses)
examples = []
for i in sorted_losses:
    if len(raw_test[i]) < 10:
        examples.append([
            ' '.join(w for w in raw_test[i] if w), 
            ' '.join(w for w in test_sents[i] if w), 
            set(predictions[i]),
            losses[i]
        ])

In [36]:
examples[-1:-10:-1]

[['...', '', {'🏘', '🐆'}, 1.0],
 ['Away !', '', {'🏘', '🐆'}, 1.0],
 ["-- RICHARDSON ' S DICTIONARY", '', {'🏘', '🐆'}, 1.0],
 ['CHAPTER 102', '', {'🏘', '🐆'}, 1.0],
 ['TAHITAN SAILOR .', '', {'🏘', '🐆'}, 1.0],
 ['" Oars !', '', {'🏘', '🐆'}, 1.0],
 ['MEANTIME THE SKY DARKENS -- THE WIND RISES .)', '', {'🏘', '🐆'}, 1.0],
 ['-- GENESIS .', '', {'🏘', '🐆'}, 1.0],
 ['Hussey ."', '', {'🏘', '🐆'}, 1.0]]

In [38]:
counter = Counter()
for p in predictions:
    s = set(p)
    for e in s:
        counter[e] += 1

print(counter['🏘']/len(predictions))
print(counter['🐆']/len(predictions))

0.09304174950298211
0.9992047713717693
